In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import bs4
import pandas as pd

import requests
import re
import time
import random

# enData

## Retrieve film data

### Get FID

In [4]:
url = 'https://www.endata.com.cn/BoxOffice/MovieStock/movies.html'

In [5]:
w = webdriver.Edge(executable_path='msedgedriver.exe')
w.get(url)

In [6]:
# Do this year by year. Check quality after each round.
year = 2021

In [241]:
df_lst = []
w.find_element_by_id('Year').send_keys(year)
time.sleep(4 + random.random() * 2)
w.find_element_by_id('searchM').click()
time.sleep(2 + random.random())
next_page = w.find_element_by_class_name('layui-laypage-next')

In [242]:
while True:
    content = w.page_source
    bs = BeautifulSoup(content)
    lst = bs.find_all('div', {'class': 'info'})

    names, fids, boxs = [], [], []
    for l in lst:
        names.append(l.find_all('a')[0].text)
        p0 = re.compile('(?<=id=)\d+')
        fids.append(p0.findall(l.find_all('a')[0].get_attribute_list('href')[0])[0])
        tmp = re.findall('(?<=id=)\d+', str(l))
        if tmp: boxs.append(tmp[0])
        else: boxs.append(float('nan'))
    df_lst.append(pd.DataFrame({'FilmName': names, 'enFID': fids, 'BoxOffice': boxs}))
    time.sleep(4 + random.random() * 2)
    if 'layui-disabled' not in next_page.get_attribute('class'):
        next_page.click()
        time.sleep(2 + random.random())
        next_page = w.find_element_by_class_name('layui-laypage-next')
    else:
        break
print('Finish this year')

Finish this year


In [243]:
len(df_lst)

12

In [244]:
df = pd.concat(df_lst)
df.shape
df.isna().sum()
df.head()
df.tail()

(113, 3)

FilmName      0
enFID         0
BoxOffice    18
dtype: int64

,FilmName,enFID,BoxOffice
0,你好，李焕英,662746,538693
1,唐人街探案3,676314,451844
2,疯狂的外星人,638300,221425
3,刺杀小说家,644881,103427
4,夺冠,676043,83569


,FilmName,enFID,BoxOffice
8,潜艇总动员：地心游记,697409,NaN
9,无翅飞翔,706646,NaN
0,无翅飞翔,706646,NaN
1,神探伍士德,669086,NaN
2,小秦归乡记,673617,NaN


In [245]:
df.to_csv(f'enFID_{year}.csv', index=False)

### Combine list

In [246]:
df_lst = [None for _ in range(2015, 2022)]

for year in range(2015, 2022):
    df = pd.read_csv(f'enFID_{year}.csv')
    df_lst[year - 2015] = df

dfm = pd.concat(df_lst)
dfm.shape

(2477, 3)

In [247]:
dfm.sample(5)

,FilmName,enFID,BoxOffice
370,QQ女人心,595611,2.0
364,骆驼客2：箭在弦,646985,1.0
142,恐怖理发店,640184,503.0
244,大魔法师孟兜兜,638734,31.0
136,江南,693366,511.0


In [251]:
dfm.sort_values('BoxOffice', ascending=False, inplace=True)
dfm.reset_index(drop=True, inplace=True)

In [252]:
dfm.head(20)

,FilmName,enFID,BoxOffice
0,战狼2,641515,568781.0
1,你好，李焕英,662746,538693.0
2,哪吒之魔童降世,662685,503502.0
3,流浪地球,642412,468680.0
4,唐人街探案3,676314,451844.0
5,红海行动,655823,365121.0
6,唐人街探案2,663419,339769.0
7,美人鱼,626153,339718.0
8,我和我的祖国,691481,317612.0
9,八佰,669412,310232.0


In [254]:
dfm.to_csv('enFID_2015_2021.csv', index=False)

### Parse page

In [13]:
df = pd.read_csv('enFID_2015_2021.csv')
df.shape
# df = df.head(10)

df[['Type', 'Format', 'Length', 'PremiereTime', 'Region', 'PublisherName', 'PublisherID', 'Directors', 'Actors', 'Producers', 'Publishers']] = None

df.head()

(2477, 3)

,FilmName,enFID,BoxOffice,Type,Format,Length,PremiereTime,Region,PublisherName,PublisherID,Directors,Actors,Producers,Publishers
0,战狼2,641515,568781.0,None,None,None,None,None,None,None,None,None,None,None
1,你好，李焕英,662746,538693.0,None,None,None,None,None,None,None,None,None,None,None
2,哪吒之魔童降世,662685,503502.0,None,None,None,None,None,None,None,None,None,None,None
3,流浪地球,642412,468680.0,None,None,None,None,None,None,None,None,None,None,None
4,唐人街探案3,676314,451844.0,None,None,None,None,None,None,None,None,None,None,None


In [4]:
w = webdriver.Edge('msedgedriver.exe')

In [ ]:
def parse_film_page(fid, browser):

    url = f'https://www.endata.com.cn/BoxOffice/MovieStock/movieShow.html?id={fid}'
    browser.get(url)
    time.sleep(2)
    content = w.page_source

    r1 = BeautifulSoup(content).find_all('div', {'class': 'bt-info'})
    tmp = r1[0].contents
    attrs = [None for _ in range(7)]

    for x in tmp:
        if str(x) == ' ':
            continue
        if '类型：' in str(x):
            attrs[0] = str(x).replace('<p>类型：', '').replace('</p>', '')
        if '制式: ' in str(x):
            attrs[1] = str(x).replace('<p>制式: ', '').replace('</p>', '')
        if '片长: ' in str(x):
            attrs[2] = str(x).replace('<p>片长: ', '').replace('</p>', '')
        if '上映时间：' in str(x):
            attrs[3] = str(x).replace('<p>上映时间：', '').replace('</p>', '')
        if '国家及地区：' in str(x):
            attrs[4] = str(x).replace('<p>国家及地区：', '').replace('</p>', '')
        if '发行公司：' in str(x):
            attrs[5] = x.text.replace('发行公司： ', '')
        if '发行公司：' in str(x):
            t = re.findall('(?<=id=)\d*', str(x))
            if t: attrs[6] = t[0]
            else: attrs[6] = '-99'

    p0 = re.compile("(?<=导演\</dt\>).*(?=\<dt\>演员)")
    directors = BeautifulSoup(p0.findall(content)[0]).find_all("dd")

    dir_name, dir_id = [], []
    for d in directors:
        dir_name.append(d.string)
        tmp = re.findall('(?<=id=)\d+', str(d))
        if tmp: dir_id.append(tmp[0])
        else: dir_id.append('-99')

    p1 = re.compile("(?<=演员\</dt\>).*(?=\<dt\>制作公司)")
    actors = BeautifulSoup(p1.findall(content)[0]).find_all("dd")

    act_name, act_id = [], []
    for a in actors:
        act_name.append(a.string)
        tmp = re.findall('(?<=id=)\d+', str(a))
        if tmp: act_id.append(tmp[0])
        else: act_id.append('-99')

    p2 = re.compile("(?<=制作公司\</dt\>).*(?=\<dt\>发行公司)")
    prods = BeautifulSoup(p2.findall(content)[0]).find_all("dd")

    prod_name, prod_id = [], []
    for p in prods:
        prod_name.append(p.string)
        tmp = re.findall('(?<=id=)\d+', str(p))
        if tmp: prod_id.append(tmp[0])
        else: prod_id.append('-99')

    p3 = re.compile("(?<=发行公司\</dt\>).*(?=\</dl\>)")
    issus = BeautifulSoup(p3.findall(content)[0]).find_all("dd")

    issu_name, issu_id = [], []
    for i in issus:
        issu_name.append(i.string)
        tmp = re.findall('(?<=id=)\d+', str(i))
        if tmp: issu_id.append(tmp[0])
        else: issu_id.append('-99')

    attrs.append(','.join(dir_id))
    attrs.append(','.join(act_id))
    attrs.append(','.join(prod_id))
    attrs.append(','.join(issu_id))
    
    return attrs

In [5]:
for idx in df.index:
    
    if df.loc[idx, 'Type']:
        print(f"{df.loc[idx, 'FilmName']} already done, skip.")
        continue

    fid = df.loc[idx, 'enFID']
    attrs = parse_film_page(fid, w)

    df.loc[idx, ['Type', 'Format', 'Length', 'PremiereTime', 
                 'Region', 'PublisherName', 'PublisherID', 
                 'Directors', 'Actors', 'Producers', 'Publishers']] = attrs

In [6]:
df.head()

,FilmName,enFID,BoxOffice,Type,Format,Length,PremiereTime,Region,PublisherName,PublisherID,Directors,Actors,Producers,Publishers
0,战狼2,641515,568781.0,"动作,战争,军事","2D,3D,IMAX",123min,2017-7-27（中国）,中国,北京京西文化旅游股份有限公司,169817,8448,"8448,208038,29636,1864045,1888479,2173,1863532...","143507,170293","159942,165606,160755,163938,161873,166383,169817"
1,你好，李焕英,662746,538693.0,"亲情,喜剧",2D,128min,2021-2-12（中国）,中国,天津猫眼微影文化传媒有限公司,175763,19353,"19353,1867670,2180209,2022988,2204213,2339982,...",169011,"175763,183883,6,162487,182873,169817,173155,15..."
2,哪吒之魔童降世,662685,503502.0,"动画,喜剧,玄幻,动作","3D,IMAX",110min,2019-7-26（中国）,中国,北京光线影业有限公司,887,2341782,"2357593,2338628,2338627,2194168,2287037",179202,"887,138681"
3,流浪地球,642412,468680.0,"科幻,冒险","2D,3D,IMAX",125min,2019-2-5（中国）,中国,中国电影集团公司,6,"2013197,2227031","8448,2286806,6623,9969,2193106,2251005,2189236...","169817,177885","169817,6,159942,161873,178956,178957,176403,14..."
4,唐人街探案3,676314,451844.0,"喜剧,动作,悬疑,侦探","2D,IMAX",136min,2021-2-12（中国）,中国,万达影视传媒有限公司,147827,"1865864,2376920","2201578,2036,156531,967537,156591,3826,27211,7...",-99,"160755,6,161873,182097,147827,184121"


In [12]:
df.to_excel('film_test.xlsx', index=False)

## Retrieve person data

### Get PID

In [2]:
df = pd.read_excel('film_test.xlsx')
df.shape
df.head(3)

(10, 14)

,FilmName,enFID,BoxOffice,Type,Format,Length,PremiereTime,Region,PublisherName,PublisherID,Directors,Actors,Producers,Publishers
0,战狼2,641515,568781,"动作,战争,军事","2D,3D,IMAX",123min,2017-7-27（中国）,中国,北京京西文化旅游股份有限公司,169817,8448,"8448,208038,29636,1864045,1888479,2173,1863532...","143507,170293","159942,165606,160755,163938,161873,166383,169817"
1,你好，李焕英,662746,538693,"亲情,喜剧",2D,128min,2021-2-12（中国）,中国,天津猫眼微影文化传媒有限公司,175763,19353,"19353,1867670,2180209,2022988,2204213,2339982,...",169011,"175763,183883,6,162487,182873,169817,173155,15..."
2,哪吒之魔童降世,662685,503502,"动画,喜剧,玄幻,动作","3D,IMAX",110min,2019-7-26（中国）,中国,北京光线影业有限公司,887,2341782,"2357593,2338628,2338627,2194168,2287037",179202,"887,138681"


In [5]:
pids = set()
for dr in df['Directors']:
    pids |= set(dr.split(','))
for act in df['Actors']:
    pids |= set(act.split(','))
len(pids)

359

In [7]:
with open('pids.txt', 'w+') as f:
    f.writelines('\n'.join(pids))

### Parse page

In [175]:
with open('pids.txt', 'r') as f:
    pids = f.readlines()
pids = [x.replace('\n', '') for x in pids]
len(pids)

359

In [176]:
df = pd.DataFrame({'PID': pids})
df[['CName', 'EName', 'Nationality', 'BirthDate', 'Height', 'Constellation', 'Bibliography']] = None
df.shape
df.head(2)

(359, 8)

,PID,CName,EName,Nationality,BirthDate,Height,Constellation,Bibliography
0,2238245,None,None,None,None,None,None,None
1,2390259,None,None,None,None,None,None,None


In [182]:
# df = df.sample(10)
# df

In [14]:
w = webdriver.Edge('msedgedriver.exe')

In [178]:
def parse_person_page(pid, browser):

    url = f'https://www.endata.com.cn/BoxOffice/MovieStock/peopleShow.html?id={pid}'
    w.get(url)

    time.sleep(2)
    content = w.page_source

    r1 = BeautifulSoup(content).find_all('div', {'class': 'left-rbox'})
    tmp = r1[0].contents
    attrs = [None for _ in range(6)]

    p0 = re.compile('(?<=<h3>).*(?=</h3>)')
    p1 = re.compile('(?<=<p class="en">).*(?=</p>)')

    for x in tmp:
        if x == ' ':
            continue

        x = str(x)
        if 'h3' in x:
            attrs[0] = re.compile('(?<=<h3>).*?(?=</h3>)').findall(x)[0]
        if 'class="en"' in x:
            attrs[1] = re.compile('(?<=<p class="en">).*(?=</p>)').findall(x)[0]
        if '国籍：' in x:
            attrs[2] = re.compile('(?<=国籍：).*?(?=</p>)').findall(x)[0]
        if '生日：' in x:
            attrs[3] = re.compile('(?<=生日：).*?(?=</p>)').findall(x)[0]
        if '身高：' in x:
            attrs[4] = re.compile('(?<=身高：).*?(?=</p>)').findall(x)[0]
        if '星座：' in x:
            attrs[5] = re.compile('(?<=星座：).*?(?=</p>)').findall(x)[0]

    r2 = BeautifulSoup(content).find_all('div', {'id': 'AboutBox'})
    if r2:
        attrs.append(re.sub('<.*?>', '', str(r2[0])))
    else:
        attrs.append('')
        
    return attrs

In [179]:
for idx in df.index:
    pid = df.loc[idx, 'PID']
    
    if df.loc[idx, 'CName']:
        print(f'{pid} has finished. Skip.')
        continue
    
    attrs = parse_person_page(pid, w)
    
    df.loc[idx, ['CName', 'EName', 'Nationality', 'BirthDate', 'Height', 'Constellation', 'Bibliography']] = attrs

In [180]:
df

,PID,CName,EName,Nationality,BirthDate,Height,Constellation,Bibliography
302,2380908,迈克尔·德拉姆戈尔德,Michael Drumgold,,,,,暂无数据
15,19353,贾玲,Ling Jia,中国,1982年4月29日,166,金牛座,贾玲，原名贾裕玲。1982年4月29日出生于湖北襄阳，毕业于中央戏剧学院。喜剧女演员，师从冯...
13,2403804,贾文田,Wentian Jia,,,,,暂无数据
336,2376920,彭瑛朝,Penny,,,,,暂无数据
347,967537,托尼·贾,Tony Jaa,,,,,暂无数据
224,1868057,海清,Christina Hai,中国,1978年1月12日,167,水瓶座,海清，本名黄怡，1977年1月12日生于中国江苏省南京市，演员，毕业于北京电影学院表演系。2...
287,33611,火火,Ka Wing Lee,中国香港,-年-月-日,,,暂无数据
124,2274707,徐轸轸,Zhenzhen Xu,,,,,暂无数据
171,1866101,张馨月,Xinyue Zhang,,,,,暂无数据
114,12276,管虎,Hu Guan,中国,1968年8月-日,190,狮子座,北京人。1991年毕业于北京电影学院导演系。\n\n 管虎作为新生代导演具有初生牛犊不...


In [181]:
df.to_excel('person_text.xlsx', index=False)

## Retrieve company data

### Get CID

In [107]:
df = pd.read_excel('film_test.xlsx')
df.shape
df.head(3)

(10, 14)

,FilmName,enFID,BoxOffice,Type,Format,Length,PremiereTime,Region,PublisherName,PublisherID,Directors,Actors,Producers,Publishers
0,战狼2,641515,568781,"动作,战争,军事","2D,3D,IMAX",123min,2017-7-27（中国）,中国,北京京西文化旅游股份有限公司,169817,8448,"8448,208038,29636,1864045,1888479,2173,1863532...","143507,170293","159942,165606,160755,163938,161873,166383,169817"
1,你好，李焕英,662746,538693,"亲情,喜剧",2D,128min,2021-2-12（中国）,中国,天津猫眼微影文化传媒有限公司,175763,19353,"19353,1867670,2180209,2022988,2204213,2339982,...",169011,"175763,183883,6,162487,182873,169817,173155,15..."
2,哪吒之魔童降世,662685,503502,"动画,喜剧,玄幻,动作","3D,IMAX",110min,2019-7-26（中国）,中国,北京光线影业有限公司,887,2341782,"2357593,2338628,2338627,2194168,2287037",179202,"887,138681"


In [115]:
cids = set()
for pub in df['PublisherID']:
    cids |= set(str(pub).split(','))
for prod in df['Producers']:
    cids |= set(prod.split(','))
for pub in df['Publishers']:
    cids |= set(pub.split(','))
len(cids)

65

In [116]:
with open('cids.txt', 'w+') as f:
    f.writelines('\n'.join(cids))

### Parse page

In [166]:
with open('cids.txt', 'r') as f:
    cids = f.readlines()
cids = [x.replace('\n', '') for x in cids]
len(cids)

65

In [168]:
df = pd.DataFrame({'CID': cids})
df[['CName', 'EName', 'Type', 'Productions', 'Introduction']] = None
df.shape
df.head(2)

(65, 6)

,CID,CName,EName,Type,Productions,Introduction
0,177293,None,None,None,None,None
1,173480,None,None,None,None,None


In [174]:
# df = df.sample(5)
# df

In [135]:
w = webdriver.Edge('msedgedriver.exe')

In [170]:
def parse_person_page(cid, browser):

    url = f'https://www.endata.com.cn/BoxOffice/MovieStock/companyShow.html?id={cid}'
    w.get(url)

    time.sleep(5)
    content = w.page_source

    r1 = BeautifulSoup(content).find_all('div', {'class': 'left-rbox'})
    tmp = r1[0].contents
    attrs = [None for _ in range(4)]

    p0 = re.compile('(?<=<h3>).*(?=</h3>)')
    p1 = re.compile('(?<=<p class="en">).*(?=</p>)')

    for x in tmp:
        if x == ' ':
            continue

        x = str(x)
        if 'h3' in x:
            attrs[0] = re.compile('(?<=<h3>).*?(?=</h3>)').findall(x)[0]
        if 'class="en"' in x:
            attrs[1] = re.compile('(?<=<p class="en">).*(?=</p>)').findall(x)[0]
        if '公司类型：' in x:
            attrs[2] = re.compile('(?<=公司类型：).*?(?=</p>)').findall(x)[0]
        if '电影作品数：' in x:
            attrs[3] = re.compile('(?<=电影作品数：).*?(?=</p>)').findall(x)[0]
#     attrs

    r2 = BeautifulSoup(content).find_all('dl', {'id': 'AboutBox'})
    if r2:
        attrs.append(re.sub('<.*?>', '', str(r2[0])))
    else:
        attrs.append('')
        
    return attrs

In [171]:
for idx in df.index:
    cid = df.loc[idx, 'CID']
    
    if df.loc[idx, 'CName']:
        print(f'{cid} has finished. Skip.')
        continue
    
    attrs = parse_person_page(cid, w)
    
    df.loc[idx, ['CName', 'EName', 'Type', 'Productions', 'Introduction']] = attrs

In [172]:
df

,CID,CName,EName,Type,Productions,Introduction
5,164106,上海腾讯影业文化传播有限公司,"Shanghai Tencent Pictures Co.,Ltd",制作公司/发行公司/出品公司/版权所有/联合出品,54,简介: 暂无介绍 联系方式: 地址： -- 电话： -- 邮箱： --
57,178957,北京影拓星瀚网络科技有限公司,Beijing Yingtuo Xinghan Network technology co....,发行公司/联合发行,4,简介: 暂无介绍 联系方式: 地址： -- 电话： -- 邮箱： --
55,165519,黑蚂蚁（上海）影业有限公司,"Black Ant Shanghai Entertainment Co.,Ltd",制作公司/发行公司/出品公司,21,简介: 暂无介绍 联系方式: 地址： -- 电话： -- 邮箱： --
25,156441,索尼发行公司,Sony Pictures Releasing,发行公司/制作公司,259,简介: 暂无介绍 联系方式: 地址： -- 电话： -- 邮箱： --
3,163938,影都文化投资发展有限公司,"Yingdu Culture Investment Development Co.,Ltd",投资公司/制作公司/发行公司/出品公司,3,简介: 暂无介绍 联系方式: 地址： -- 电话： -- 邮箱： --


In [173]:
df.to_excel('company_text.xlsx', index=False)

# Douban

## Selenium

In [48]:
df = pd.read_csv('test.csv')
df.shape
df = df.sample(10)
df[['dbName', 'dbURL', 'dbScore', 'dbViewer']] = None
df

(2364, 17)

,FilmName,enFID,BoxOffice,Type,Format,Length,PremiereTime,Region,PublisherName,PublisherID,Directors,DirectorID,Actors,ActorID,Producers,ProducerID,Publishers,dbName,dbURL,dbScore,dbViewer
490,不败雄心,661445,1.0,"剧情,动作,运动",2D,93min,2017-6-30（中国）,中国,上海乐从和影业有限公司,169774.0,10896,NaN,"1948632,1866145,2014392",NaN,"158785,157186,169773",NaN,169774,None,None,None,None
2260,东方欲晓,630884,NaN,"革命,纪录片",2D,NaN,2015-11-17（中国）,中国,NaN,NaN,1866658,NaN,"2191364,2362368,1867369,2180758",NaN,"148186,532",NaN,-99,None,None,None,None
881,漂亮的村事,672381,11.0,剧情,2D,90min,2018-9-16（中国）,中国,NaN,NaN,"2010001,13617",NaN,"13617,2359627,2395278,1865860,1105,1867128,1020",NaN,169257,NaN,-99,None,None,None,None
447,过昭关,673173,38.0,"剧情,儿童,家庭",2D,93min,2019-5-20（中国）,中国,江苏安石英纳电影发行有限公司,163896.0,2179505,NaN,"2352607,2352608,2385676,2385677",NaN,174164,NaN,163896,None,None,None,None
1933,乘风破浪,655798,104853.0,剧情,2D,102min,2017-1-28（中国）,中国,上海博纳文化传媒有限公司,167691.0,"1877325,2277012",NaN,"5302,8663,1865823,2185005,1862658,2360366,2261...",NaN,"164065,160286,159524,144459,160390,161459,1618...",NaN,"167691,138681",None,None,None,None
1295,恋爱兄弟,642686,NaN,"爱情,农村",2D,84分钟,2015-9-17（中国）,中国,中国农业电影电视中心,143830.0,"2244170,2210891",NaN,"1860567,1866504,2244180,2244181",NaN,-99,NaN,143830,None,None,None,None
1928,识色，幸也,640399,49.0,"爱情,剧情",2D,103min,2017-11-3（中国）,中国,天津龙行文化传播有限公司,170222.0,2022821,NaN,"1865182,1166,2205024,2175508,2373,2204448",NaN,"162369,161276,170222",NaN,"170222,166181,166054,173024",None,None,None,None
2348,八仙之各显神通,680430,NaN,"奇幻,冒险",2D,NaN,2019（中国）,中国,NaN,NaN,24694,NaN,-99,NaN,"160995,162229",NaN,-99,None,None,None,None
1401,马小乐之玩具也疯狂,643746,742.0,动画,"2D,3D",79min,2016-4-30（中国）,中国,北京环鹰时代文化传媒有限公司,160921.0,690441,NaN,"2227394,2185872,2385227,2200415,2177316,2175273",NaN,"143195,159950",NaN,160921,None,None,None,None
159,中邪,660754,42.0,"惊悚,恐怖",2D,97min,2018-12（中国）,中国,天津猫眼文化传媒有限公司,162770.0,2279710,NaN,"2288838,2193272,2288839,2288840,2288841,228884...",NaN,"162770,169103,175218,165802,169636,169972",NaN,"162770,169103",None,None,None,None


In [4]:
title = '谍·莲花'
url = f'https://search.douban.com/movie/subject_search?search_text={title}'

In [5]:
w = webdriver.Edge('msedgedriver.exe')

In [51]:
def parse_db_search(fname, browser):

    url = f'https://search.douban.com/movie/subject_search?search_text={fname}'
    w.get(url)

    time.sleep(2)
    content = w.page_source

    r = BeautifulSoup(content).find_all('div', {'class': 'detail'})
    db_title = r[0].find('a', {'class': 'title-text'}).string.split('\u200e')[0]
    db_url = r[0].find('a', {'class': 'title-text'}).attrs['href']
    
    tmp = re.findall('\d+', r[0].find('span', {'class': 'pl'}).string)
    if tmp:
        db_viewer = float(tmp[0])
        db_score = float(r[0].find('span', {'class': 'rating_nums'}).string)
    else:
        db_viewer = float('nan')
        db_score = float('nan')
        
    return [db_title, db_url, db_score, db_viewer]

In [52]:
for idx in df.index:
    fname = df.loc[idx, 'FilmName']
    
    if df.loc[idx, 'dbName']:
        print(f'{fname} has finished. Skip.')
        continue
    
    df.loc[idx, ['dbName', 'dbURL', 'dbScore', 'dbViewer']] = parse_db_search(fname, w)

In [53]:
df

,FilmName,enFID,BoxOffice,Type,Format,Length,PremiereTime,Region,PublisherName,PublisherID,Directors,DirectorID,Actors,ActorID,Producers,ProducerID,Publishers,dbName,dbURL,dbScore,dbViewer
490,不败雄心,661445,1.0,"剧情,动作,运动",2D,93min,2017-6-30（中国）,中国,上海乐从和影业有限公司,169774.0,10896,NaN,"1948632,1866145,2014392",NaN,"158785,157186,169773",NaN,169774,不败雄心,https://movie.douban.com/subject/27075182/,NaN,NaN
2260,东方欲晓,630884,NaN,"革命,纪录片",2D,NaN,2015-11-17（中国）,中国,NaN,NaN,1866658,NaN,"2191364,2362368,1867369,2180758",NaN,"148186,532",NaN,-99,东方欲晓,https://movie.douban.com/subject/34982249/,NaN,NaN
881,漂亮的村事,672381,11.0,剧情,2D,90min,2018-9-16（中国）,中国,NaN,NaN,"2010001,13617",NaN,"13617,2359627,2395278,1865860,1105,1867128,1020",NaN,169257,NaN,-99,漂亮的村事,https://movie.douban.com/subject/27181026/,NaN,NaN
447,过昭关,673173,38.0,"剧情,儿童,家庭",2D,93min,2019-5-20（中国）,中国,江苏安石英纳电影发行有限公司,163896.0,2179505,NaN,"2352607,2352608,2385676,2385677",NaN,174164,NaN,163896,过昭关,https://movie.douban.com/subject/30206431/,7.9,14142.0
1933,乘风破浪,655798,104853.0,剧情,2D,102min,2017-1-28（中国）,中国,上海博纳文化传媒有限公司,167691.0,"1877325,2277012",NaN,"5302,8663,1865823,2185005,1862658,2360366,2261...",NaN,"164065,160286,159524,144459,160390,161459,1618...",NaN,"167691,138681",乘风破浪,https://movie.douban.com/subject/26862259/,6.8,509114.0
1295,恋爱兄弟,642686,NaN,"爱情,农村",2D,84分钟,2015-9-17（中国）,中国,中国农业电影电视中心,143830.0,"2244170,2210891",NaN,"1860567,1866504,2244180,2244181",NaN,-99,NaN,143830,恋爱兄弟,https://movie.douban.com/subject/26969709/,NaN,NaN
1928,识色，幸也,640399,49.0,"爱情,剧情",2D,103min,2017-11-3（中国）,中国,天津龙行文化传播有限公司,170222.0,2022821,NaN,"1865182,1166,2205024,2175508,2373,2204448",NaN,"162369,161276,170222",NaN,"170222,166181,166054,173024",识色，幸也,https://movie.douban.com/subject/26882309/,3.3,1140.0
2348,八仙之各显神通,680430,NaN,"奇幻,冒险",2D,NaN,2019（中国）,中国,NaN,NaN,24694,NaN,-99,NaN,"160995,162229",NaN,-99,八仙之各显神通,https://movie.douban.com/subject/26277338/,NaN,NaN
1401,马小乐之玩具也疯狂,643746,742.0,动画,"2D,3D",79min,2016-4-30（中国）,中国,北京环鹰时代文化传媒有限公司,160921.0,690441,NaN,"2227394,2185872,2385227,2200415,2177316,2175273",NaN,"143195,159950",NaN,160921,马小乐之玩具也疯狂,https://movie.douban.com/subject/26630850/,3.4,340.0
159,中邪,660754,42.0,"惊悚,恐怖",2D,97min,2018-12（中国）,中国,天津猫眼文化传媒有限公司,162770.0,2279710,NaN,"2288838,2193272,2288839,2288840,2288841,228884...",NaN,"162770,169103,175218,165802,169636,169972",NaN,"162770,169103",中邪,https://movie.douban.com/subject/26820833/,6.8,48657.0


In [55]:
df.to_excel('test_dbinfo.xlsx', index=False)

## Requests [TODO]

If do in `requests` way, need to decrypt the results. Work-in-progress.